In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import datetime
import gc
import gc
import os
import random
import xgboost as xgb
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.cluster import FeatureAgglomeration

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge

from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle

In [34]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')
sub=pd.read_csv('SampleSubmission.csv')

In [35]:
pd.set_option('display.max_columns', None)

Shifting **Features** 

In [36]:



train['precipitable_water_entire_atmosphere123'] = train.temperature_2m_above_ground.shift(periods=2)
test['precipitable_water_entire_atmosphere123'] = test.temperature_2m_above_ground.shift(periods=2)
train['precipitable_water_entire_atmosphere12'] = train.temperature_2m_above_ground.shift(periods=3)
test['precipitable_water_entire_atmosphere12'] = test.temperature_2m_above_ground.shift(periods=3)
#temperature_2m_above_ground	u_component_of_wind_10m_above_ground
train['precipitable_water_entire_atmosphere av hier'] = train.precipitable_water_entire_atmosphere.shift(periods=2)
train['precipitable_water_entire_atmosphere av av hier'] = train.precipitable_water_entire_atmosphere.shift(periods=3)
train['precipitable_water_entire_atmosphere av av av hier'] = train.precipitable_water_entire_atmosphere.shift(periods=4)

test['precipitable_water_entire_atmosphere av hier'] = test.precipitable_water_entire_atmosphere.shift(periods=2)
test['precipitable_water_entire_atmosphere av av hier'] = test.precipitable_water_entire_atmosphere.shift(periods=3)

test['precipitable_water_entire_atmosphere av av av hier'] = test.precipitable_water_entire_atmosphere.shift(periods=4)

In [37]:
print(train.shape)
train.head()

(30557, 87)


,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,precipitable_water_entire_atmosphere123,precipitable_water_entire_atmosphere12,precipitable_water_entire_atmosphere av hier,precipitable_water_entire_atmosphere av av hier,precipitable_water_entire_atmosphere av av av hier
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,22.379054,-0.000010,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118,NaN,NaN,NaN,NaN,NaN
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,33.089495,0.000114,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.00015

In [38]:
train['L3_NO2_NO2_column_number_density2']=train['L3_NO2_NO2_column_number_density'].shift(periods=2)
train['L3_O3_O3_column_number_density2']=train['L3_O3_O3_column_number_density'].shift(periods=2)
train['L3_HCHO_tropospheric_HCHO_column_number_density2']=train['L3_HCHO_tropospheric_HCHO_column_number_density'].shift(periods=2)
train['L3_CO_CO_column_number_density2']=train['L3_CO_CO_column_number_density'].shift(periods=2)
train['L3_CLOUD_surface_albedo2']=train['L3_CLOUD_surface_albedo'].shift(periods=2)
train['L3_CLOUD_cloud_optical_depth2']=train['L3_CLOUD_cloud_optical_depth'].shift(periods=2)

train['L3_NO2_NO2_column_number_density3']=train['L3_NO2_NO2_column_number_density'].shift(periods=3)
train['L3_O3_O3_column_number_density3']=train['L3_O3_O3_column_number_density'].shift(periods=3)
train['L3_HCHO_tropospheric_HCHO_column_number_density3']=train['L3_HCHO_tropospheric_HCHO_column_number_density'].shift(periods=3)
train['L3_CO_CO_column_number_density3']=train['L3_CO_CO_column_number_density'].shift(periods=3)
train['L3_CLOUD_surface_albedo3']=train['L3_CLOUD_surface_albedo'].shift(periods=3)
train['L3_CLOUD_cloud_optical_depth3']=train['L3_CLOUD_cloud_optical_depth'].shift(periods=3)


test['L3_NO2_NO2_column_number_density3']=test['L3_NO2_NO2_column_number_density'].shift(periods=3)
test['L3_O3_O3_column_number_density3']=test['L3_O3_O3_column_number_density'].shift(periods=3)
test['L3_HCHO_tropospheric_HCHO_column_number_density3']=test['L3_HCHO_tropospheric_HCHO_column_number_density'].shift(periods=3)
test['L3_CO_CO_column_number_density3']=test['L3_CO_CO_column_number_density'].shift(periods=3)
test['L3_CLOUD_surface_albedo3']=test['L3_CLOUD_surface_albedo'].shift(periods=3)
test['L3_CLOUD_cloud_optical_depth3']=test['L3_CLOUD_cloud_optical_depth'].shift(periods=3)

test['L3_NO2_NO2_column_number_density2']=test['L3_NO2_NO2_column_number_density'].shift(periods=2)
test['L3_O3_O3_column_number_density2']=test['L3_O3_O3_column_number_density'].shift(periods=2)
test['L3_HCHO_tropospheric_HCHO_column_number_density2']=test['L3_HCHO_tropospheric_HCHO_column_number_density'].shift(periods=2)
test['L3_CO_CO_column_number_density2']=test['L3_CO_CO_column_number_density'].shift(periods=2)
test['L3_CLOUD_surface_albedo2']=test['L3_CLOUD_surface_albedo'].shift(periods=2)
test['L3_CLOUD_cloud_optical_depth2']=test['L3_CLOUD_cloud_optical_depth'].shift(periods=2)

In [39]:
train.shape,test.shape

((30557, 99), (16136, 94))

In [40]:
#using data of yesterday 

In [41]:
78%94

78

In [42]:
l = [np.NaN]
s=train['precipitable_water_entire_atmosphere'][0:len(train)-1]
for i in s :
  l.append(i)
train['pr hier']=l
l = [np.NaN]
s=test['precipitable_water_entire_atmosphere'][0:len(test)-1]
for i in s :
  l.append(i)
test['pr hier']=l

In [43]:
l = [np.NaN]
s=train['temperature_2m_above_ground'][0:len(train)-1]
for i in s :
  l.append(i)
train['temp hier']=l
l = [np.NaN]
s=test['temperature_2m_above_ground'][0:len(test)-1]
for i in s :
  l.append(i)
test['temp hier']=l

In [44]:
for i in range (len(test)):
  if i%94==0 :
    test['pr hier'][i]=np.NaN
for i in range (len(train)):
  if i%94==0 :
    train['pr hier'][i]=np.NaN

C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\2872310948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pr hier'][i]=np.NaN
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\2872310948.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pr hier'][i]=np.NaN


In [45]:
train.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,precipitable_water_entire_atmosphere123,precipitable_water_entire_atmosphere12,precipitable_water_entire_atmosphere av hier,precipitable_water_entire_atmosphere av av hier,precipitable_water_entire_atmosphere av av av hier,L3_NO2_NO2_column_number_density2,L3_O3_O3_column_number_density2,L3_HCHO_tropospheric_HCHO_column_number_density2,L3_CO_CO_column_number_density2,L3_CLOUD_surface_albedo2,L3_CLOUD_cloud_optical_depth2,L3_NO2_NO2_column_number_density3,L3_O3_O3_column_number_density3,L3_HCHO_tropospheric_HCHO_column_number_density3,L3_CO_CO_column_number_density3,L3_CLOUD_surface_albedo3,L3_CLOUD_cloud_optical_depth3,pr hier,temp hier
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,22.379054,-0.000010,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912

In [46]:
for i in range (len(test)):
  if i%94==0 :
    test['temp hier'][i]=np.NaN
for i in range (len(train)):
  if i%94==0 :
    train['temp hier'][i]=np.NaN

C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\374520549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['temp hier'][i]=np.NaN
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\374520549.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['temp hier'][i]=np.NaN


In [47]:
l = [np.NaN]
s=train['relative_humidity_2m_above_ground'][0:len(train)-1]
for i in s :
  l.append(i)
train['HUMID hier']=l
l = [np.NaN]
s=test['relative_humidity_2m_above_ground'][0:len(test)-1]
for i in s :
  l.append(i)
test['HUMID hier']=l
for i in range (len(test)):
  if i%94==0 : 
    test['HUMID hier'][i]=np.NaN
for i in range (len(train)):
  if i%94==0 :
    train['HUMID hier'][i]=np.NaN

C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\1531108047.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['HUMID hier'][i]=np.NaN
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\1531108047.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['HUMID hier'][i]=np.NaN


Drop columns which are not importants

In [48]:
test=test.drop(columns=[
 
 'L3_HCHO_sensor_azimuth_angle',
 'L3_HCHO_sensor_zenith_angle',
 'L3_HCHO_solar_azimuth_angle',
 'L3_HCHO_solar_zenith_angle',
])
train=train.drop(columns=[
                          
 
 'L3_HCHO_sensor_azimuth_angle',
 'L3_HCHO_sensor_zenith_angle',
 'L3_HCHO_solar_azimuth_angle',
 'L3_HCHO_solar_zenith_angle',
 ])
test=test.drop(columns=[
                        #'L3_CO_sensor_azimuth_angle',
 'L3_CO_sensor_zenith_angle',
 #'L3_CO_solar_azimuth_angle',
 'L3_CO_solar_zenith_angle',
 'L3_SO2_sensor_azimuth_angle',
 'L3_SO2_sensor_zenith_angle',
 'L3_SO2_solar_azimuth_angle',
 'L3_SO2_solar_zenith_angle']

)
train=train.drop(columns=[
                          #'L3_CO_sensor_azimuth_angle',
 'L3_CO_sensor_zenith_angle',
 #'L3_CO_solar_azimuth_angle',
 'L3_CO_solar_zenith_angle',
 

 'L3_SO2_sensor_azimuth_angle',
 'L3_SO2_sensor_zenith_angle',
 'L3_SO2_solar_azimuth_angle',
 'L3_SO2_solar_zenith_angle']                                

 )

In [49]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table


In [50]:
p=[]
l=list(test.columns)
l.append('target')
for c in list(train.columns):
  if c not in l:
    p.append(c)
    train.drop(c,inplace=True,axis=1)

In [51]:
[col for col in train.columns if not train[col].isnull().any()]

['Place_ID X Date',
 'Date',
 'Place_ID',
 'target',
 'precipitable_water_entire_atmosphere',
 'relative_humidity_2m_above_ground',
 'specific_humidity_2m_above_ground',
 'temperature_2m_above_ground',
 'u_component_of_wind_10m_above_ground',
 'v_component_of_wind_10m_above_ground']

In [52]:
result_train=missing_zero_values_table(train)

Your selected dataframe has 88 columns and 30557 Rows.
There are 78 columns that have missing values.


In [53]:
k=['L3_CH4_solar_zenith_angle','L3_CH4_solar_azimuth_angle','L3_CH4_sensor_zenith_angle','L3_CH4_sensor_azimuth_angle','L3_CH4_aerosol_optical_depth','L3_CH4_aerosol_height','L3_CH4_CH4_column_volume_mixing_ratio_dry_air']
train.drop(k,inplace=True,axis=1)
test.drop(k,inplace=True,axis=1)

In [54]:

h=['L3_NO2_NO2_column_number_density', 	'L3_NO2_NO2_slant_column_number_density', 	'L3_NO2_absorbing_aerosol_index', 	'L3_NO2_cloud_fraction', 	'L3_NO2_sensor_altitude', 	'L3_NO2_sensor_azimuth_angle', 	'L3_NO2_sensor_zenith_angle', 	'L3_NO2_solar_azimuth_angle' 	,'L3_NO2_solar_zenith_angle', 	'L3_NO2_stratospheric_NO2_column_number_density', 	'L3_NO2_tropopause_pressure', 	'L3_NO2_tropospheric_NO2_column_number_density']

In [55]:


r=['L3_CLOUD_cloud_base_height', 	'L3_CLOUD_cloud_base_pressure' ,	'L3_CLOUD_cloud_fraction', 	'L3_CLOUD_cloud_optical_depth', 	'L3_CLOUD_cloud_top_height', 	'L3_CLOUD_cloud_top_pressure', 	'L3_CLOUD_sensor_azimuth_angle', 	'L3_CLOUD_sensor_zenith_angle', 	'L3_CLOUD_solar_azimuth_angle', 	'L3_CLOUD_solar_zenith_angle', 	'L3_CLOUD_surface_albedo']

In [56]:
p=['L3_AER_AI_absorbing_aerosol_index' ,	'L3_AER_AI_sensor_altitude', 	'L3_AER_AI_sensor_azimuth_angle' ,	'L3_AER_AI_sensor_zenith_angle', 	'L3_AER_AI_solar_azimuth_angle' 	,'L3_AER_AI_solar_zenith_angle']

**Extracting** **Informations** ***from*** **date**

In [57]:
train["Date"] = pd.to_datetime(train['Date'])
test["Date"] = pd.to_datetime(test['Date'])

def applyer(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0 






def makedate(df):
      
   df['day of week']=df['Date'].dt.dayofweek
   df['weekend'] = df['Date'].apply(applyer)
   df['month']=0
   df['day']=0
   df['winter']=0
   for i in range(len(df)):  
           
           df['month'][i]=df['Date'][i].month 
           df['day'][i]=df['Date'][i].day
           if (df['month'][i]==1 or df['month'][i]==2 ):
             df['winter'][i]=1

makedate(train)
makedate(test)


#test["day"] = test.Date.dt.day
#train["day"] = train.Date.dt.day

#train["month"] = train.Date.dt.month

#test["month"] = test.Date.dt.month

#train["day"] = train.Date.dt.dayofweek

#test["day"] = test.Date.dt.dayofweek

train["quarter"] = train.Date.dt.quarter

test["quarter"] = test.Date.dt.quarter


C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\2186771530.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'][i]=df['Date'][i].month
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\2186771530.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day'][i]=df['Date'][i].day
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\2186771530.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['winter'][i]=1
C:\Users\DELL\Ap

## Fillna

In [58]:
train=train.fillna(method='ffill')
train=train.fillna(train.mean())
test=test.fillna(method ='ffill')
test=test.fillna(test.mean())

C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\1801765937.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  train=train.fillna(train.mean())
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\1801765937.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train=train.fillna(train.mean())
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\1801765937.py:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  test=test.fillna(test.mean())
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\1801765937.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version th

In [59]:
#mean_temp

In [60]:
air_temperature_filler = pd.DataFrame(train.groupby(['Place_ID','month'])['temperature_2m_above_ground'].mean())
train['mean temp per month']=0
for i in range (len(train)):
  
  train['mean temp per month'][i]=air_temperature_filler.loc[(train['Place_ID'][i], train['month'][i]), :]

air_temperature_filler = pd.DataFrame(test.groupby(['Place_ID','month'])['temperature_2m_above_ground'].mean())

test['mean temp per month']=0
for i in range (len(test)):
  
  test['mean temp per month'][i]=air_temperature_filler.loc[(test['Place_ID'][i], test['month'][i]), :]


C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\2510742257.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['mean temp per month'][i]=air_temperature_filler.loc[(train['Place_ID'][i], train['month'][i]), :]
C:\Users\DELL\AppData\Local\Temp\ipykernel_22152\2510742257.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean temp per month'][i]=air_temperature_filler.loc[(test['Place_ID'][i], test['month'][i]), :]


# PCA

In [61]:
pca = PCA(random_state=42,n_components=1)
pg_features =  train.filter(regex='L3_AER_.*')
train_pca = pca.fit_transform(pg_features)
train['pca_AE'] = train_pca[:,0]
pg_features =  test.filter(regex='L3_AER.*')
test_pca = pca.transform(pg_features)
test['pca_AE'] = test_pca[:,0]
pca = PCA(random_state=42,n_components=1)
pg_features =  train.filter(regex='L3_NO2_.*')
train_pca = pca.fit_transform(pg_features)
train['pca_NO'] = train_pca[:,0]
pg_features =  test.filter(regex='L3_NO2.*')
test_pca = pca.transform(pg_features)
test['pca_NO'] = test_pca[:,0]


C:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [62]:
trainag=train.copy()
testag=test.copy()

In [63]:
train.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
test.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
target=train['target']
train.drop('target',axis=1,inplace=True)

In [78]:
feature=['precipitable_water_entire_atmosphere', 	'relative_humidity_2m_above_ground', 	'specific_humidity_2m_above_ground', 	'temperature_2m_above_ground', 	'u_component_of_wind_10m_above_ground', 	'v_component_of_wind_10m_above_ground', 	'L3_NO2_NO2_column_number_density', 'L3_SO2_SO2_column_number_density',	'L3_SO2_SO2_column_number_density_amf' 	,'L3_SO2_SO2_slant_column_number_density', 	 'L3_NO2_tropospheric_NO2_column_number_density', 'L3_HCHO_tropospheric_HCHO_column_number_density', 'L3_CO_CO_column_number_density', 'L3_HCHO_HCHO_slant_column_number_density'	,'L3_CO_H2O_column_number_density'	,'L3_O3_O3_column_number_density'	,'L3_NO2_NO2_slant_column_number_density']
train1=train.copy()
test1=test.copy()
for c in list(feature):
  if c not in l:
    train1.drop(c,inplace=True,axis=1)
    test1.drop(c,inplace=True,axis=1)
    
pca = PCA(random_state=42,n_components=1)
train_pca = pca.fit_transform(train1)
train['pca_feature'] = train_pca[:,0]
test_pca = pca.transform(test1)
test['pca_feature'] = test_pca[:,0]

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
train.shape,test.shape

In [ ]:
train.head()

In [65]:

from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
dev_X, val_X, dev_y, val_y = train_test_split(train, target, test_size = 0.15, random_state = 42)


In [66]:
#FIRST MODEL

In [67]:
#without agg
!pip install catboost
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=30000,
                             learning_rate=0.045,
                             depth=8,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=300)
cb_model.fit(train, target,
             use_best_model=True,
             verbose=50)

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 46.1192759	total: 344ms	remaining: 2h 52m 8s
50:	learn: 32.6605581	total: 7.76s	remaining: 1h 15m 54s
100:	learn: 29.8207796	total: 14.6s	remaining: 1h 12m 6s
150:	learn: 28.0519631	total: 22.4s	remaining: 1h 13m 38s
200:	learn: 26.8226198	total: 28.9s	remaining: 1h 11m 22s
250:	learn: 25.7163283	total: 35.1s	remaining: 1h 9m 20s
300:	learn: 24.6777455	total: 41.4s	remaining: 1h 8m 8s
350:	learn: 23.7081421	total: 47.7s	remaining: 1h 7m 6s
400:	learn: 22.9037824	total: 54.4s	remaining: 1h 6m 53s
450:	learn: 22.1785531	total: 1m 1s	remaining: 1h 7m 6s
500:	learn: 21.5484593	total: 1m 7s	remaining: 1h 6m 40s
550:	learn: 20.9762777	total: 1m 14s	remaining: 1h 6m 12s
600:	learn: 20.3954855	total: 1m 21s	remaining: 1h 6m 37s
650:	learn: 19.8771077	total: 1m 29s	remaining: 1h 6m 52s
700:	learn: 19.4077649	total: 1m 35s	remaining: 1h 6m 52s
750:	learn: 18.9470617	total: 1m 42s	remaining: 1h 6m 34s
800:	learn: 18.4963621	total: 1m 49s	remaining: 1h 6m 18s
850:	learn: 18.0812382	total

7250:	learn: 2.9823112	total: 15m 35s	remaining: 48m 54s
7300:	learn: 2.9504002	total: 15m 41s	remaining: 48m 47s
7350:	learn: 2.9165294	total: 15m 48s	remaining: 48m 42s
7400:	learn: 2.8826467	total: 15m 54s	remaining: 48m 34s
7450:	learn: 2.8505679	total: 16m	remaining: 48m 27s
7500:	learn: 2.8214734	total: 16m 7s	remaining: 48m 21s
7550:	learn: 2.7893543	total: 16m 13s	remaining: 48m 14s
7600:	learn: 2.7575958	total: 16m 19s	remaining: 48m 7s
7650:	learn: 2.7283664	total: 16m 26s	remaining: 48m
7700:	learn: 2.7012082	total: 16m 32s	remaining: 47m 53s
7750:	learn: 2.6711050	total: 16m 38s	remaining: 47m 46s
7800:	learn: 2.6392844	total: 16m 45s	remaining: 47m 40s
7850:	learn: 2.6106765	total: 16m 52s	remaining: 47m 35s
7900:	learn: 2.5826949	total: 16m 58s	remaining: 47m 28s
7950:	learn: 2.5560494	total: 17m 4s	remaining: 47m 21s
8000:	learn: 2.5294775	total: 17m 10s	remaining: 47m 14s
8050:	learn: 2.4992744	total: 17m 17s	remaining: 47m 7s
8100:	learn: 2.4717471	total: 17m 23s	remai

14450:	learn: 0.6713392	total: 31m 4s	remaining: 33m 26s
14500:	learn: 0.6654543	total: 31m 11s	remaining: 33m 20s
14550:	learn: 0.6595694	total: 31m 18s	remaining: 33m 14s
14600:	learn: 0.6534714	total: 31m 25s	remaining: 33m 8s
14650:	learn: 0.6474129	total: 31m 32s	remaining: 33m 2s
14700:	learn: 0.6411995	total: 31m 40s	remaining: 32m 57s
14750:	learn: 0.6358907	total: 31m 47s	remaining: 32m 51s
14800:	learn: 0.6301130	total: 31m 55s	remaining: 32m 47s
14850:	learn: 0.6243060	total: 32m 2s	remaining: 32m 41s
14900:	learn: 0.6187379	total: 32m 10s	remaining: 32m 36s
14950:	learn: 0.6125035	total: 32m 18s	remaining: 32m 30s
15000:	learn: 0.6070315	total: 32m 25s	remaining: 32m 25s
15050:	learn: 0.6016327	total: 32m 32s	remaining: 32m 19s
15100:	learn: 0.5962498	total: 32m 38s	remaining: 32m 12s
15150:	learn: 0.5905628	total: 32m 45s	remaining: 32m 6s
15200:	learn: 0.5848235	total: 32m 52s	remaining: 32m
15250:	learn: 0.5794127	total: 32m 58s	remaining: 31m 53s
15300:	learn: 0.5741470

21600:	learn: 0.1797476	total: 46m 37s	remaining: 18m 7s
21650:	learn: 0.1781633	total: 46m 44s	remaining: 18m 1s
21700:	learn: 0.1766344	total: 46m 51s	remaining: 17m 55s
21750:	learn: 0.1751157	total: 46m 57s	remaining: 17m 48s
21800:	learn: 0.1736968	total: 47m 3s	remaining: 17m 41s
21850:	learn: 0.1722077	total: 47m 9s	remaining: 17m 35s
21900:	learn: 0.1706354	total: 47m 16s	remaining: 17m 28s
21950:	learn: 0.1691867	total: 47m 23s	remaining: 17m 22s
22000:	learn: 0.1676748	total: 47m 30s	remaining: 17m 16s
22050:	learn: 0.1663227	total: 47m 38s	remaining: 17m 10s
22100:	learn: 0.1648601	total: 47m 45s	remaining: 17m 4s
22150:	learn: 0.1635380	total: 47m 51s	remaining: 16m 57s
22200:	learn: 0.1621078	total: 47m 57s	remaining: 16m 50s
22250:	learn: 0.1606238	total: 48m 4s	remaining: 16m 44s
22300:	learn: 0.1591423	total: 48m 10s	remaining: 16m 37s
22350:	learn: 0.1576185	total: 48m 17s	remaining: 16m 31s
22400:	learn: 0.1562564	total: 48m 23s	remaining: 16m 24s
22450:	learn: 0.1548

28800:	learn: 0.0510765	total: 58m 9s	remaining: 2m 25s
28850:	learn: 0.0506510	total: 58m 13s	remaining: 2m 19s
28900:	learn: 0.0502334	total: 58m 19s	remaining: 2m 13s
28950:	learn: 0.0497999	total: 58m 23s	remaining: 2m 6s
29000:	learn: 0.0493945	total: 58m 27s	remaining: 2m
29050:	learn: 0.0489817	total: 58m 31s	remaining: 1m 54s
29100:	learn: 0.0485650	total: 58m 38s	remaining: 1m 48s
29150:	learn: 0.0481542	total: 58m 46s	remaining: 1m 42s
29200:	learn: 0.0477618	total: 58m 51s	remaining: 1m 36s
29250:	learn: 0.0473842	total: 58m 55s	remaining: 1m 30s
29300:	learn: 0.0469925	total: 58m 59s	remaining: 1m 24s
29350:	learn: 0.0466007	total: 59m 4s	remaining: 1m 18s
29400:	learn: 0.0462046	total: 59m 11s	remaining: 1m 12s
29450:	learn: 0.0458156	total: 59m 16s	remaining: 1m 6s
29500:	learn: 0.0454045	total: 59m 20s	remaining: 1m
29550:	learn: 0.0450151	total: 59m 24s	remaining: 54.2s
29600:	learn: 0.0446113	total: 59m 29s	remaining: 48.1s
29650:	learn: 0.0442172	total: 59m 33s	remain

In [68]:
pred_test_cat2 = (cb_model.predict(test))


In [69]:
#train['L3_SO2_SO2_column_number_density1'] = train.L3_SO2_SO2_column_number_density.shift(periods=1)
#test['L3_SO2_SO2_column_number_density1'] = train.L3_SO2_SO2_column_number_density.shift(periods=1)

#train['L3_NO2_tropospheric_NO2_column_number_density1'] = train.specific_humidity_2m_above_ground.shift(periods=1)
#test['L3_NO2_tropospheric_NO2_column_number_density1'] = test.specific_humidity_2m_above_ground.shift(periods=1)

In [70]:
#dont run this xD
#train['relative_humidity_2m_above_ground1'] = train.relative_humidity_2m_above_ground.shift(periods=1)
#test['relative_humidity_2m_above_ground1'] = train.relative_humidity_2m_above_ground.shift(periods=1)

#train['specific_humidity_2m_above_ground1'] = train.specific_humidity_2m_above_ground.shift(periods=1)
#test['specific_humidity_2m_above_ground1'] = test.specific_humidity_2m_above_ground.shift(periods=1)

#train['temperature_2m_above_ground1'] = train.temperature_2m_above_ground.shift(periods=1)
#test['temperature_2m_above_ground1'] = train.temperature_2m_above_ground.shift(periods=1)

#train['u_component_of_wind_10m_above_ground1'] = train.u_component_of_wind_10m_above_ground.shift(periods=1)
#test['u_component_of_wind_10m_above_ground1'] = test.u_component_of_wind_10m_above_ground.shift(periods=1)

#train['v_component_of_wind_10m_above_ground1'] = train.v_component_of_wind_10m_above_ground.shift(periods=1)
#test['v_component_of_wind_10m_above_ground1'] = train.v_component_of_wind_10m_above_ground.shift(periods=1)

#train['precipitable_water_entire_atmosphere1'] = train.precipitable_water_entire_atmosphere.shift(periods=1)
#test['precipitable_water_entire_atmosphere1'] = test.precipitable_water_entire_atmosphere.shift(periods=1)

In [71]:
trainag.head()

,Place_ID X Date,Date,Place_ID,target,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_solar_azimuth_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,precipitable_water_entire_atmosphere123,precipitable_water_entire_atmosphere12,precipitable_water_entire_atmosphere av hier,precipitable_water_entire_atmosphere av av hier,precipitable_water_entire_atmosphere av av av hier,L3_NO2_NO2_column_number_density2,L3_O3_O3_column_number_density2,L3_HCHO_tropospheric_HCHO_column_number_density2,L3_CO_CO_column_number_density2,L3_CLOUD_surface_albedo2,L3_CLOUD_cloud_optical_depth2,L3_NO2_NO2_column_number_density3,L3_O3_O3_column_number_density3,L3_HCHO_tropospheric_HCHO_column_number_density3,L3_CO_CO_column_number_density3,L3_CLOUD_surface_albedo3,L3_CLOUD_cloud_optical_depth3,pr hier,temp hier,HUMID hier,day of week,weekend,month,day,winter,quarter,mean temp per month,pca_AE,pca_NO
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,-61.789016,-0.000010,0.000000,0.000064,0.566828,2505.218548,77542.072128,0.000000,17.950029,3280.496698,70847.513552,76.536426,38.593017,-61.752587,22.363665,0.226013,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,9.321311,9.321471,15.302323,15.302591,15.302886,0.000096,0.138800,0.000083,0.036513,0.226013,17.950566,0.000096,0.138799,0.000083,0.036513,0.226014,17.950811,15.305183,9.323654,70.551504,3,0,1,2,1,1,NaN,7658.699707,-83286.588393
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,-74.457583,0.000114,0.059433,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.000050,-1.452612,0.059433,9

 #using agg

In [72]:
aggs = {}
aggs['L3_HCHO_tropospheric_HCHO_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_CO_CO_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_O3_O3_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_NO2_NO2_slant_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_NO2_NO2_column_number_density'] = ['sum','max','min','mean','std']

aggs['L3_O3_cloud_fraction'] = ['max','min','mean','std']
aggs['L3_CO_H2O_column_number_density'] = ['sum','max','min','mean','std']
aggs['L3_HCHO_HCHO_slant_column_number_density'] = ['sum','max','min','mean','std']


aggs['relative_humidity_2m_above_ground'] = ['sum','max','min','mean','std']
aggs['u_component_of_wind_10m_above_ground'] = ['sum','max','min','mean','std']


aggs['precipitable_water_entire_atmosphere'] = ['sum','max','min','mean','std']
aggs['specific_humidity_2m_above_ground'] = ['sum','max','min','mean','std']

aggs['v_component_of_wind_10m_above_ground'] = ['sum','max','min','mean','std']
aggs['temperature_2m_above_ground'] = ['sum','max','min','mean','std']


aggs['pca_AE'] = ['sum','max','min','mean','std']


In [73]:
agg_trans = trainag.groupby(['Place_ID']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (trainag.groupby('Place_ID')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

agg_trans = pd.merge(df, agg_trans, on='Place_ID', how='left')
trainag = pd.merge(trainag,agg_trans, on='Place_ID', how='left')
trainag.head()

,Place_ID X Date,Date,Place_ID,target,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_solar_azimuth_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,precipitable_water_entire_atmosphere123,precipitable_water_entire_atmosphere12,precipitable_water_entire_atmosphere av hier,precipitable_water_entire_atmosphere av av hier,precipitable_water_entire_atmosphere av av av hier,L3_NO2_NO2_column_number_density2,L3_O3_O3_column_number_density2,L3_HCHO_tropospheric_HCHO_column_number_density2,L3_CO_CO_column_number_density2,L3_CLOUD_surface_albedo2,L3_CLOUD_cloud_optical_depth2,L3_NO2_NO2_column_number_density3,L3_O3_O3_column_number_density3,L3_HCHO_tropospheric_HCHO_column_number_density3,L3_CO_CO_column_number_density3,L3_CLOUD_surface_albedo3,L3_CLOUD_cloud_optical_depth3,pr hier,temp hier,HUMID hier,day of week,weekend,month,day,winter,quarter,mean temp per month,pca_AE,pca_NO,1transactions_count,L3_HCHO_tropospheric_HCHO_column_number_density_sum,L3_HCHO_tropospheric_HCHO_column_number_density_max,L3_HCHO_tropospheric_HCHO_column_number_density_min,L3_HCHO_tropospheric_HCHO_column_number_density_mean,L3_HCHO_tropospheric_HCHO_column_number_density_std,L3_CO_CO_column_number_density_sum,L3_CO_CO_column_number_density_max,L3_CO_CO_column_number_density_min,L3_CO_CO_column_number_density_mean,L3_CO_CO_column_number_density_std,L3_O3_O3_column_number_density_sum,L3_O3_O3_column_number_density_max,L3_O3_O3_column_number_density_min,L3_O3_O3_column_number_density_mean,L3_O3_O3_column_number_density_std,L3_NO2_NO2_slant_column_number_density_sum,L3_NO2_NO2_slant_column_number_density_max,L3_NO2_NO2_slant_column_number_density_min,L3_NO2_NO2_slant_column_number_density_mean,L3_NO2_NO2_slant_column_number_density_std,L3_NO2_NO2_column_number_density_sum,L3_NO2_NO2_column_number_density_max,L3_NO2_NO2_column_number_density_min,L3_NO2_NO2_column_number_density_mean,L3_NO2_NO2_column_number_density_std,L3_O3_cloud_fraction_max,L3_O3_cloud_fraction_min,L3_O3_cloud_fraction_mean,L3_O3_cloud_fraction_std,L3_CO_H2O_column_number_density_sum,L3_CO_H2O_column_number_density_max,L3_CO_H2O_column_number_density_min,L3_CO_H2O_column_number_density_mean,L3_CO_H2O_column_number_density_std,L3_HCHO_HCHO_slant_column_number_density_sum,L3_HCHO_HCHO_slant_column_number_density_max,L3_HCHO_HCHO_slant_column_number_density_min,L3_HCHO_HCHO_slant_column_number_densit

In [74]:
agg_trans = testag.groupby(['Place_ID']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (testag.groupby('Place_ID')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

agg_trans = pd.merge(df, agg_trans, on='Place_ID', how='left')
testag = pd.merge(testag,agg_trans, on='Place_ID', how='left')
testag.head()

,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_solar_azimuth_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,precipitable_water_entire_atmosphere123,precipitable_water_entire_atmosphere12,precipitable_water_entire_atmosphere av hier,precipitable_water_entire_atmosphere av av hier,precipitable_water_entire_atmosphere av av av hier,L3_NO2_NO2_column_number_density3,L3_O3_O3_column_number_density3,L3_HCHO_tropospheric_HCHO_column_number_density3,L3_CO_CO_column_number_density3,L3_CLOUD_surface_albedo3,L3_CLOUD_cloud_optical_depth3,L3_NO2_NO2_column_number_density2,L3_O3_O3_column_number_density2,L3_HCHO_tropospheric_HCHO_column_number_density2,L3_CO_CO_column_number_density2,L3_CLOUD_surface_albedo2,L3_CLOUD_cloud_optical_depth2,pr hier,temp hier,HUMID hier,day of week,weekend,month,day,winter,quarter,mean temp per month,pca_AE,pca_NO,1transactions_count,L3_HCHO_tropospheric_HCHO_column_number_density_sum,L3_HCHO_tropospheric_HCHO_column_number_density_max,L3_HCHO_tropospheric_HCHO_column_number_density_min,L3_HCHO_tropospheric_HCHO_column_number_density_mean,L3_HCHO_tropospheric_HCHO_column_number_density_std,L3_CO_CO_column_number_density_sum,L3_CO_CO_column_number_density_max,L3_CO_CO_column_number_density_min,L3_CO_CO_column_number_density_mean,L3_CO_CO_column_number_density_std,L3_O3_O3_column_number_density_sum,L3_O3_O3_column_number_density_max,L3_O3_O3_column_number_density_min,L3_O3_O3_column_number_density_mean,L3_O3_O3_column_number_density_std,L3_NO2_NO2_slant_column_number_density_sum,L3_NO2_NO2_slant_column_number_density_max,L3_NO2_NO2_slant_column_number_density_min,L3_NO2_NO2_slant_column_number_density_mean,L3_NO2_NO2_slant_column_number_density_std,L3_NO2_NO2_column_number_density_sum,L3_NO2_NO2_column_number_density_max,L3_NO2_NO2_column_number_density_min,L3_NO2_NO2_column_number_density_mean,L3_NO2_NO2_column_number_density_std,L3_O3_cloud_fraction_max,L3_O3_cloud_fraction_min,L3_O3_cloud_fraction_mean,L3_O3_cloud_fraction_std,L3_CO_H2O_column_number_density_sum,L3_CO_H2O_column_number_density_max,L3_CO_H2O_column_number_density_min,L3_CO_H2O_column_number_density_mean,L3_CO_H2O_column_number_density_std,L3_HCHO_HCHO_slant_column_number_density_sum,L3_HCHO_HCHO_slant_column_number_density_max,L3_HCHO_HCHO_slant_column_number_density_min,L3_HCHO_HCHO_slant_column_number_density_mean,

In [75]:
trainag.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
testag.drop(['Place_ID X Date','Date','Place_ID'],inplace=True,axis=1)
target=trainag['target']
trainag.drop('target',axis=1,inplace=True)

In [76]:
feature=['precipitable_water_entire_atmosphere', 	'relative_humidity_2m_above_ground', 	'specific_humidity_2m_above_ground', 	'temperature_2m_above_ground', 	'u_component_of_wind_10m_above_ground', 	'v_component_of_wind_10m_above_ground', 	'L3_NO2_NO2_column_number_density', 'L3_SO2_SO2_column_number_density',	'L3_SO2_SO2_column_number_density_amf' 	,'L3_SO2_SO2_slant_column_number_density', 	 'L3_NO2_tropospheric_NO2_column_number_density', 'L3_HCHO_tropospheric_HCHO_column_number_density', 'L3_CO_CO_column_number_density', 'L3_HCHO_HCHO_slant_column_number_density'	,'L3_CO_H2O_column_number_density'	,'L3_O3_O3_column_number_density'	,'L3_NO2_NO2_slant_column_number_density']
train1=trainag.copy()
test1=testag.copy()
for c in list(feature):
  if c not in l:
    train1.drop(c,inplace=True,axis=1)
    test1.drop(c,inplace=True,axis=1)
    
pca = PCA(random_state=42,n_components=1)
train_pca = pca.fit_transform(train1)
trainag['pca_feature'] = train_pca[:,0]
test_pca = pca.transform(test1)
testag['pca_feature'] = test_pca[:,0]

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# modeling

In [ ]:
#!pip install catboost

from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=30000,
                             learning_rate=0.045,
                             depth=8,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=300)
cb_model.fit(trainag,target,
             use_best_model=True,
             verbose=50)

In [ ]:
pred_test_cat = (cb_model.predict(testag))


In [ ]:
#Ensembling

In [ ]:
x=(pred_test_cat2*0.4+pred_test_cat*0.6)

In [77]:
results_df = pd.DataFrame({"Place_ID X Date":sub['Place_ID X Date'], "target": np.clip(x, 0, a_max=None)})

results_df.to_csv("emino.csv", index=False)

In [ ]:
results_df.head()

In [ ]:
           #0.2,0.6,0.2